# The interesting topics in this year

In [1]:
import pandas as pd

def filter_by_topic(df, topic):
    df['google_scholar_link'] = df['title'].apply(lambda x: 'https://scholar.google.com/scholar?q=' + x.replace(' ', '+'))
    df = df.drop_duplicates(subset=['title'])
    return df[df['topic'].str.contains(topic, case=False, na=False)].copy()

In [2]:
from semanticscholar import SemanticScholar
import time
from tqdm import tqdm
tqdm.pandas()

def get_paper_citation(title):
    sch = SemanticScholar()
    time.sleep(1)
    title = title.replace('-', ' ')
    try:
        papers = sch.search_paper(title)
        if len(papers) > 0:
            return papers[0].citationCount
    except:
        print(f'Error in get citation for {title}')
    
    return 0
    

def get_topic_citations(df):
    df['ss_citations'] = df['title'].progress_apply(get_paper_citation)
    return df

In [3]:
def get_conf_topics(conf_path='data/2023/5_iclr.csv_topics.csv', topic='multimodal'):
    df_conf = pd.read_csv(conf_path)
    df_conf_topic = filter_by_topic(df_conf, topic=topic)
    print(len(df_conf_topic), len(df_conf))
    df_conf_topic = get_topic_citations(df_conf_topic)
    return df_conf_topic[['title','year','source','ss_citations','google_scholar_link']]

In [4]:
def get_conf_topics_all(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal']):
    df_conf = pd.read_csv(conf_path)

    save_path = conf_path + '_emerging.xlsx'
    writer = pd.ExcelWriter(save_path, engine="xlsxwriter")

    for topic in topics:
        print(f"topic {topic}")
        df_conf_topic = get_conf_topics(conf_path, topic)
        df_conf_topic.to_excel(writer, sheet_name=topic, index=False)
    writer.close()
    print(f'Saved to {save_path}')

## 2023, ICLR, May

In [6]:
# get_conf_topics_all(conf_path='data/2023/5_iclr.csv_topics.csv', topics=['multimodal', 'diffusion models', 'llm', 'self-supervision'])

## 2023, CVPR, June

In [7]:
# get_conf_topics_all(conf_path='data/2023/6_cvpr.csv_topics.csv', topics=['3d', 'zero_few-shot', 'transformer', 'segmentation', 'multimodal', 
# 'imitation learning', 'radiance field', 'motion', 'diffusion models', 'multi-agent'])

## 2023, ICML, July

In [5]:
# get_conf_topics_all(conf_path='data/2023/7_icml.csv_topics.csv', topics=['zero_few-shot', 'graph', 'optimization', 'reinforcement learning', 'representation',
# 'motion', 'decision trees', 'radiance field', 'diffusion models', 'federated learning'])

## 2023, ICCV, Oct.

In [5]:
# get_conf_topics_all(conf_path='data/2023/10_ICCV.csv_topics.csv', topics=['object detection', '3d', 'segmentation', 'transformer', 'zero_few-shot'])

## 2023, NIPS, Dec.

In [6]:
# get_conf_topics_all(conf_path='data/2023/12_nips.csv_topics.csv', topics=['object detection', 'zero_few-shot', 'graph', 'optimization', 'llm', 'radiance field', 'motion', 'differential equation solving', 'rl'])

# 2024, ICLR, May

In [8]:
# get_conf_topics_all(conf_path='data/2024/5_iclr.csv_topics.csv', topics=['graph', 'zero_few-shot', 'llm', 'transformer', 'generative model', 'reward model', 'watermark', 'foundation models', 'model merging', 'diffusion models'])